In [2]:
!pip install chromadb sentence-transformers transformers torch pandas langchain_text_splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.9/475.9 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 22.7 MB/s 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import json
import glob
import pandas as pd
import chromadb
import torch
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [37]:
# ===== SETUP =====
print("Setting up ChromaDB...")
client = chromadb.Client()
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

collection = client.create_collection(
    name="cooking_assistant",
    embedding_function=sentence_transformer_ef
)

Setting up ChromaDB...


In [36]:
# Use this when rerunning the code
# client = chromadb.PersistentClient(path="/content/drive/MyDrive/final-project/chroma_db")

# Delete and recreate in one go
client.delete_collection(name="cooking_assistant")

In [23]:
# Load LLM (only once at startup)
print("Loading language model (this may take a minute)...")

# Check device availability: CUDA (NVIDIA) > MPS (Apple Silicon) > CPU
if torch.cuda.is_available():
    device = 0  # CUDA GPU
    device_name = 'CUDA GPU'
elif torch.backends.mps.is_available():
    device = torch.device('mps')  # Apple Silicon GPU
    device_name = 'MPS (Apple Silicon GPU)'
else:
    device = -1  # CPU
    device_name = 'CPU'

generator = pipeline(
    'text-generation',
    model='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    device=device
)
print(f"Model loaded on: {device_name}")

Loading language model (this may take a minute)...


Device set to use cuda:0


KeyboardInterrupt: 

In [38]:
# ===== NEW CHUNKING FUNCTIONS =====

def chunk_fcs_json(data, source_file):
    chunks = []

    # Introduction
    chunks.append({
        'text': f"{data['title']}\n\n{data['introduction']}",
        'metadata': {'type': 'introduction', 'source': source_file}
    })

    # Cooking methods and techniques
    for method_name, method_data in data['cooking_methods'].items():
        if isinstance(method_data, dict) and 'techniques' in method_data:
            for technique_name, technique_desc in method_data['techniques'].items():
                chunks.append({
                    'text': f"Cooking Method: {method_name.replace('_', ' ').title()}\n\n"
                            f"{method_data['description']}\n\n"
                            f"Technique: {technique_name}\n{technique_desc}",
                    'metadata': {
                        'type': 'technique',
                        'method': method_name,
                        'technique': technique_name,
                        'source': source_file
                    }
                })
        else:
            chunks.append({
                'text': f"Cooking Method: {method_name.title()}\n\n{method_data['description']}",
                'metadata': {'type': 'method', 'method': method_name, 'source': source_file}
            })

    # Kitchen tools
    if 'kitchen_tools' in data:
        for category, category_data in data['kitchen_tools'].items():
            for tool_name, tool_desc in category_data['items'].items():
                chunks.append({
                    'text': f"Kitchen Tool: {tool_name}\n\n{tool_desc}",
                    'metadata': {
                        'type': 'kitchen_tool',
                        'category': category,
                        'tool': tool_name,
                        'source': source_file
                    }
                })

    return chunks


def chunk_healthy_cooking(data, source_file):
    chunks = []

    for method_name, method_desc in data['cooking_methods'].items():
        chunks.append({
            'text': f"Healthy Cooking Method: {method_name}\n\n{method_desc}",
            'metadata': {
                'type': 'healthy_method',
                'method': method_name.lower(),
                'source': source_file
            }
        })

    if 'food_preparation_tips' in data:
        tips_text = "Healthy Food Preparation Tips:\n\n" + "\n\n".join(
            f"• {tip}" for tip in data['food_preparation_tips']
        )
        chunks.append({
            'text': tips_text,
            'metadata': {'type': 'preparation_tips', 'source': source_file}
        })

    return chunks


def chunk_recipe_json(data, source_file):
    ingredients_text = "\n".join(
        f"- {ing['item']}: {ing['quantity']}"
        for ing in data['ingredients']
    )

    recipe_text = f"""Recipe: {data['dish_name']}

Prep time: {data.get('prep_time', 'N/A')}
Cooking time: {data.get('cooking_time', 'N/A')}
Portions: {data.get('portions', 'N/A')}

Ingredients:
{ingredients_text}"""

    return [{
        'text': recipe_text,
        'metadata': {
            'type': 'recipe',
            'dish_name': data['dish_name'],
            'source': source_file
        }
    }]


def chunk_csv_simple(file_path):
    """
    Put entire CSV in one chunk (for small CSVs ~5 lines)
    """
    try:
        df = pd.read_csv(file_path)

        # Format as readable text
        text_parts = [f"Data from {file_path.split('/')[-1]}:\n"]

        for idx, row in df.iterrows():
            row_text = " | ".join([f"{col}: {val}" for col, val in row.items() if pd.notna(val)])
            text_parts.append(row_text)

        text = "\n".join(text_parts)

        return [{
            'text': text,
            'metadata': {
                'type': 'csv_data',
                'source': file_path,
                'filename': file_path.split('/')[-1],
                'rows': len(df),
                'columns': len(df.columns)
            }
        }]

    except Exception as e:
        print(f"  ✗ Error processing CSV {file_path}: {e}")
        return []


def chunk_json_simple(file_path):
    """
    Put each row of JSON in its own chunk - subject: text related to subject format
    """
    chunks = []

    try:
        with open(file_path, 'r') as f:
            data = json.load(f)

        for key, val in data.items():
            # Ensure metadata values are simple types (str, int, float, bool, None)
            # Convert dicts or lists to string representation for metadata
            metadata_value = json.dumps(val) if isinstance(val, (dict, list)) else val

            chunks.append({
                'text': f"{key}: {val}", # Original value can remain in the text content
                'metadata': {
                    'type': 'json_data',
                    'source': file_path,
                    'key': key,
                    'value': metadata_value # Store string representation in metadata
                }
            })

    except Exception as e:
        print(f"  ✗ Error processing JSON {file_path}: {e}")
        return []

    return chunks


def chunk_reddit_json(file_path):
    """
    Process Reddit JSON with posts and comments grouped together
    """
    chunks = []

    try:
        with open(file_path, 'r') as f:
            data = json.load(f)

        source_name = file_path.split('/')[-1].replace('.json', '')

        for post_idx, post in enumerate(data):
            # --- Create post chunk ---
            post_text = f"Title: {post['title']}\n\nPost: {post['selftext']}"
            chunks.append({
                'text': post_text,
                'metadata': {
                    'type': 'reddit_post',
                    'source': file_path,
                    'post_title': post['title']
                }
            })

            # --- Create comment chunks (group 3–5 comments each) ---
            comments = post.get("comments", [])
            group_size = 5

            for i in range(0, len(comments), group_size):
                group = comments[i:i+group_size]
                comment_text = f"Comments for post '{post['title']}':\n"

                for c in group:
                    comment_text += f"{c['author']}: {c['body']}\n"

                chunks.append({
                    'text': comment_text,
                    'metadata': {
                        'type': 'reddit_comments',
                        'source': file_path,
                        'post_title': post['title']
                    }
                })

        print(f"✓ Loaded {len(chunks)} chunks from Reddit data: {source_name}")

    except Exception as e:
        print(f"  ✗ Error processing Reddit JSON {file_path}: {e}")
        return []

    return chunks


def apply_recursive_chunking(chunks, chunk_size=500, chunk_overlap=50):
    """
    Apply RecursiveCharacterTextSplitter to long chunks
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", "!", "?", " "]
    )

    final_chunks = []

    for chunk in chunks:
        text_parts = splitter.split_text(chunk['text'])

        # If text was split into multiple parts
        if len(text_parts) > 1:
            for i, part in enumerate(text_parts):
                final_chunks.append({
                    'text': part,
                    'metadata': {**chunk['metadata'], 'chunk_part': i}
                })
        else:
            # Keep original chunk if it's already small enough
            final_chunks.append(chunk)

    return final_chunks


def chunk_ingredient_data_json(data, source_file):
    """
    Chunk ingredient pairing data from the flavor network info
    """
    chunks = []

    # --- 1. Individual ingredient pair chunks ---
    # Each pair becomes its own searchable chunk
    if 'ingredient_pairs' in data:
        for pair in data['ingredient_pairs']:
            ing1 = pair['ingredient1'].replace('_', ' ').title()
            ing2 = pair['ingredient2'].replace('_', ' ').title()
            shared = pair['num_shared_compound']

            # Create descriptive text for the pairing
            pairing_strength = "strongly" if shared > 30 else "moderately" if shared > 15 else "somewhat"

            text = f"""Ingredient Pairing: {ing1} and {ing2}

These ingredients share {shared} chemical compounds, suggesting they {pairing_strength} complement each other in recipes.

Pairing strength: {pairing_strength.title()}
Shared compounds: {shared}"""

            chunks.append({
                'text': text,
                'metadata': {
                    'type': 'ingredient_pairing',
                    'ingredient1': pair['ingredient1'],
                    'ingredient2': pair['ingredient2'],
                    'shared_compounds': shared,
                    'pairing_strength': pairing_strength,
                    'source': source_file
                }
            })

    # --- 2. Grouped pairing chunks by ingredient ---
    # For queries like "what pairs well with orange?"
    if 'ingredient_pairs' in data:
        # Build a lookup: ingredient -> list of pairings
        ingredient_map = {}

        for pair in data['ingredient_pairs']:
            ing1 = pair['ingredient1']
            ing2 = pair['ingredient2']
            shared = pair['num_shared_compound']

            # Add to ing1's pairing list
            if ing1 not in ingredient_map:
                ingredient_map[ing1] = []
            ingredient_map[ing1].append((ing2, shared))

            # Add to ing2's pairing list (bidirectional)
            if ing2 not in ingredient_map:
                ingredient_map[ing2] = []
            ingredient_map[ing2].append((ing1, shared))

        # Create a chunk for each ingredient's pairings
        for ingredient, pairings in ingredient_map.items():
            # Sort by shared compounds (best pairings first)
            pairings.sort(key=lambda x: x[1], reverse=True)

            ing_name = ingredient.replace('_', ' ').title()

            # Build pairing list text
            pairing_lines = []
            for paired_ing, shared in pairings[:10]:  # Top 10 pairings
                paired_name = paired_ing.replace('_', ' ').title()
                pairing_lines.append(f"  • {paired_name} ({shared} shared compounds)")

            text = f"""Ingredient Pairings for {ing_name}

Based on shared chemical compounds, {ing_name} pairs well with:

{chr(10).join(pairing_lines)}

These pairings are ranked by the number of shared flavor compounds."""

            chunks.append({
                'text': text,
                'metadata': {
                    'type': 'ingredient_pairing_summary',
                    'ingredient': ingredient,
                    'num_pairings': len(pairings),
                    'best_pairing': pairings[0][0] if pairings else None,
                    'best_pairing_compounds': pairings[0][1] if pairings else 0,
                    'source': source_file
                }
            })

    # --- 3. Individual ingredient info chunks ---
    if 'ingredients' in data:
        for ingredient, info in data['ingredients'].items():
            ing_name = ingredient.replace('_', ' ').title()
            category = info.get('category', 'unknown').title()
            prevalence = info.get('prevalence', 0)

            text = f"""Ingredient: {ing_name}

Category: {category}
Prevalence in recipes: {prevalence:.2%}

This ingredient appears in approximately {prevalence:.1%} of recipes in the database."""

            chunks.append({
                'text': text,
                'metadata': {
                    'type': 'ingredient_info',
                    'ingredient': ingredient,
                    'category': info.get('category'),
                    'prevalence': prevalence,
                    'source': source_file
                }
            })

    # --- 4. Category-based pairing chunks (optional) ---
    # Group pairings by category combinations
    if 'ingredient_pairs' in data and 'ingredients' in data:
        category_pairings = {}

        for pair in data['ingredient_pairs']:
            ing1 = pair['ingredient1']
            ing2 = pair['ingredient2']

            # Get categories
            cat1 = data['ingredients'].get(ing1, {}).get('category', 'unknown')
            cat2 = data['ingredients'].get(ing2, {}).get('category', 'unknown')

            # Create category pair key
            cat_key = tuple(sorted([cat1, cat2]))

            if cat_key not in category_pairings:
                category_pairings[cat_key] = []

            category_pairings[cat_key].append({
                'ing1': ing1.replace('_', ' ').title(),
                'ing2': ing2.replace('_', ' ').title(),
                'shared': pair['num_shared_compound']
            })

        # Create chunks for each category pairing
        for (cat1, cat2), pairings in category_pairings.items():
            if len(pairings) < 3:  # Skip categories with too few pairings
                continue

            # Get top 5 examples
            pairings.sort(key=lambda x: x['shared'], reverse=True)
            examples = pairings[:5]

            example_lines = [f"  • {p['ing1']} + {p['ing2']} ({p['shared']} compounds)"
                           for p in examples]

            text = f"""Category Pairing: {cat1.title()} with {cat2.title()}

Ingredients from these categories often pair well together. Top examples:

{chr(10).join(example_lines)}

Total pairings found: {len(pairings)}"""

            chunks.append({
                'text': text,
                'metadata': {
                    'type': 'category_pairing',
                    'category1': cat1,
                    'category2': cat2,
                    'num_pairings': len(pairings),
                    'source': source_file
                }
            })

    return chunks

def chunk_cuisine_ingredients_dict(cuisine_dict, source_file):
    """
    Chunk pre-parsed cuisine-ingredient dictionary
    Creates multiple chunk types for different retrieval patterns

    Args:
        cuisine_dict: Dictionary with cuisine as key, list of ingredient lists as value
        source_file: Path to source file for metadata
    """
    chunks = []

    try:
        # --- 1. Individual recipe chunks ---
        # Each recipe as a standalone chunk
        recipe_idx = 0
        for cuisine, recipe_list in cuisine_dict.items():
            for ingredients in recipe_list:
                ing_formatted = ', '.join([ing.replace('_', ' ').title() for ing in ingredients])

                text = f"""Cuisine: {cuisine}

Traditional {cuisine} dish using: {ing_formatted}

Number of ingredients: {len(ingredients)}"""

                chunks.append({
                    'text': text,
                    'metadata': {
                        'type': 'cuisine_recipe',
                        'cuisine': cuisine.lower(),
                        'num_ingredients': len(ingredients),
                        'ingredients': '|'.join(ingredients),  # Store as pipe-separated string
                        'source': source_file
                    }
                })
                recipe_idx += 1

        # --- 2. Cuisine summary chunks ---
        # Aggregated view per cuisine showing common ingredients
        for cuisine, recipe_list in cuisine_dict.items():
            # Count ingredient frequency across all recipes in this cuisine
            from collections import defaultdict
            ingredient_freq = defaultdict(int)
            for recipe in recipe_list:
                for ing in recipe:
                    ingredient_freq[ing] += 1

            # Sort by frequency
            sorted_ingredients = sorted(ingredient_freq.items(), key=lambda x: x[1], reverse=True)

            # Get top 10 most common ingredients
            top_ingredients = sorted_ingredients[:10]
            ing_lines = [f"  • {ing.replace('_', ' ').title()} (used in {count}/{len(recipe_list)} recipes)"
                        for ing, count in top_ingredients]

            text = f"""{cuisine} Cuisine Profile

Based on {len(recipe_list)} traditional recipes, common ingredients include:

{chr(10).join(ing_lines)}

This cuisine features {len(ingredient_freq)} unique ingredients across its dishes."""

            chunks.append({
                'text': text,
                'metadata': {
                    'type': 'cuisine_summary',
                    'cuisine': cuisine.lower(),
                    'num_recipes': len(recipe_list),
                    'num_unique_ingredients': len(ingredient_freq),
                    'most_common_ingredient': top_ingredients[0][0] if top_ingredients else None,
                    'source': source_file
                }
            })

        # --- 3. Ingredient-to-cuisine mapping chunks ---
        # For queries like "what cuisines use ginger?"
        from collections import defaultdict
        ingredient_to_cuisines = defaultdict(set)

        for cuisine, recipe_list in cuisine_dict.items():
            for recipe in recipe_list:
                for ing in recipe:
                    ingredient_to_cuisines[ing].add(cuisine)

        # Create chunks for ingredients used in multiple cuisines
        for ingredient, cuisines in ingredient_to_cuisines.items():
            if len(cuisines) >= 2:  # Only include if used in 2+ cuisines
                ing_name = ingredient.replace('_', ' ').title()
                cuisine_list = ', '.join(sorted(cuisines))

                text = f"""Ingredient: {ing_name}

Found in {len(cuisines)} cuisines: {cuisine_list}

This ingredient is versatile and appears across multiple culinary traditions."""

                chunks.append({
                    'text': text,
                    'metadata': {
                        'type': 'ingredient_cuisine_map',
                        'ingredient': ingredient,
                        'num_cuisines': len(cuisines),
                        'cuisines': '|'.join(sorted(cuisines)),
                        'source': source_file
                    }
                })

        # --- 4. Recipe similarity chunks ---
        # Group recipes by ingredient overlap for substitution suggestions
        for cuisine, recipe_list in cuisine_dict.items():
            # Find recipes with high ingredient overlap (5+ shared ingredients)
            for i, recipe1 in enumerate(recipe_list):
                for j in range(i+1, len(recipe_list)):
                    recipe2 = recipe_list[j]
                    shared = set(recipe1) & set(recipe2)

                    if len(shared) >= 5:
                        shared_formatted = ', '.join([ing.replace('_', ' ').title() for ing in sorted(shared)])

                        text = f"""{cuisine} Recipe Pairing

Two traditional {cuisine} dishes share these ingredients: {shared_formatted}

Shared ingredients: {len(shared)}

These recipes could be good to prepare together as they use similar ingredients."""

                        chunks.append({
                            'text': text,
                            'metadata': {
                                'type': 'recipe_similarity',
                                'cuisine': cuisine.lower(),
                                'shared_count': len(shared),
                                'source': source_file
                            }
                        })

        print(f"✓ Created {len(chunks)} cuisine-ingredient chunks ({recipe_idx} recipes)")
        return chunks

    except Exception as e:
        print(f"✗ Error processing cuisine data: {e}")
        return []


def chunk_cuisine_ingredients_dict(cuisine_dict, source_file):
    """
    Chunk pre-parsed cuisine-ingredient dictionary
    Creates multiple chunk types for different retrieval patterns

    Args:
        cuisine_dict: Dictionary with cuisine as key, list of ingredient lists as value
        source_file: Path to source file for metadata
    """
    chunks = []

    try:
        # --- 1. Individual recipe chunks ---
        # Each recipe as a standalone chunk
        recipe_idx = 0
        for cuisine, recipe_list in cuisine_dict.items():
            for ingredients in recipe_list:
                ing_formatted = ', '.join([ing.replace('_', ' ').title() for ing in ingredients])

                text = f"""Cuisine: {cuisine}

Traditional {cuisine} dish using: {ing_formatted}

Number of ingredients: {len(ingredients)}"""

                chunks.append({
                    'text': text,
                    'metadata': {
                        'type': 'cuisine_recipe',
                        'cuisine': cuisine.lower(),
                        'num_ingredients': len(ingredients),
                        'ingredients': '|'.join(ingredients),  # Store as pipe-separated string
                        'source': source_file
                    }
                })
                recipe_idx += 1

        # --- 2. Cuisine summary chunks ---
        # Aggregated view per cuisine showing common ingredients
        for cuisine, recipe_list in cuisine_dict.items():
            # Count ingredient frequency across all recipes in this cuisine
            from collections import defaultdict
            ingredient_freq = defaultdict(int)
            for recipe in recipe_list:
                for ing in recipe:
                    ingredient_freq[ing] += 1

            # Sort by frequency
            sorted_ingredients = sorted(ingredient_freq.items(), key=lambda x: x[1], reverse=True)

            # Get top 10 most common ingredients
            top_ingredients = sorted_ingredients[:10]
            ing_lines = [f"  • {ing.replace('_', ' ').title()} (used in {count}/{len(recipe_list)} recipes)"
                        for ing, count in top_ingredients]

            text = f"""{cuisine} Cuisine Profile

Based on {len(recipe_list)} traditional recipes, common ingredients include:

{chr(10).join(ing_lines)}

This cuisine features {len(ingredient_freq)} unique ingredients across its dishes."""

            chunks.append({
                'text': text,
                'metadata': {
                    'type': 'cuisine_summary',
                    'cuisine': cuisine.lower(),
                    'num_recipes': len(recipe_list),
                    'num_unique_ingredients': len(ingredient_freq),
                    'most_common_ingredient': top_ingredients[0][0] if top_ingredients else None,
                    'source': source_file
                }
            })

        # --- 3. Ingredient-to-cuisine mapping chunks ---
        # For queries like "what cuisines use ginger?"
        from collections import defaultdict
        ingredient_to_cuisines = defaultdict(set)

        for cuisine, recipe_list in cuisine_dict.items():
            for recipe in recipe_list:
                for ing in recipe:
                    ingredient_to_cuisines[ing].add(cuisine)

        # Create chunks for ingredients used in multiple cuisines
        for ingredient, cuisines in ingredient_to_cuisines.items():
            if len(cuisines) >= 2:  # Only include if used in 2+ cuisines
                ing_name = ingredient.replace('_', ' ').title()
                cuisine_list = ', '.join(sorted(cuisines))

                text = f"""Ingredient: {ing_name}

Found in {len(cuisines)} cuisines: {cuisine_list}

This ingredient is versatile and appears across multiple culinary traditions."""

                chunks.append({
                    'text': text,
                    'metadata': {
                        'type': 'ingredient_cuisine_map',
                        'ingredient': ingredient,
                        'num_cuisines': len(cuisines),
                        'cuisines': '|'.join(sorted(cuisines)),
                        'source': source_file
                    }
                })

        # --- 4. Recipe similarity chunks ---
        # Group recipes by ingredient overlap for substitution suggestions
        for cuisine, recipe_list in cuisine_dict.items():
            # Find recipes with high ingredient overlap (5+ shared ingredients)
            for i, recipe1 in enumerate(recipe_list):
                for j in range(i+1, len(recipe_list)):
                    recipe2 = recipe_list[j]
                    shared = set(recipe1) & set(recipe2)

                    if len(shared) >= 5:
                        shared_formatted = ', '.join([ing.replace('_', ' ').title() for ing in sorted(shared)])

                        text = f"""{cuisine} Recipe Pairing

Two traditional {cuisine} dishes share these ingredients: {shared_formatted}

Shared ingredients: {len(shared)}

These recipes could be good to prepare together as they use similar ingredients."""

                        chunks.append({
                            'text': text,
                            'metadata': {
                                'type': 'recipe_similarity',
                                'cuisine': cuisine.lower(),
                                'shared_count': len(shared),
                                'source': source_file
                            }
                        })

        print(f"✓ Created {len(chunks)} cuisine-ingredient chunks ({recipe_idx} recipes)")
        return chunks

    except Exception as e:
        print(f"✗ Error processing cuisine data: {e}")
        return []

In [40]:
# ===== NEW INGESTION =====

def ingest_all_documents(collection, apply_chunking=False):
    """
    Main ingestion function that processes all document types

    Args:
        collection: ChromaDB collection object
        apply_chunking: If True, applies RecursiveCharacterTextSplitter to all chunks
    """
    all_chunks = []

    # Processed data path
    processed_data_folder = '/content/drive/MyDrive/final-project/data/processed'

    # Load core technique files
    print("Loading core cooking technique files...")
    specific_files = {
        os.path.join(processed_data_folder, 'FCS.json'): chunk_fcs_json,
        os.path.join(processed_data_folder, 'healthy_cooking_method.json'): chunk_healthy_cooking,
    }

    for file_path, chunk_function in specific_files.items():
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            chunks = chunk_function(data, file_path)
            all_chunks.extend(chunks)
            print(f"✓ Loaded {len(chunks)} chunks from {os.path.basename(file_path)}")
        except Exception as e:
            print(f"✗ Error loading {file_path}: {e}")

    # Load all recipes from foc folder
    print("\nLoading recipes from foc folder...")
    foc_folder = os.path.join(processed_data_folder, "foc")
    recipe_count = 0

    if os.path.exists(foc_folder):
        foc_json_files = glob.glob(os.path.join(foc_folder, '*.json'))

        for file_path in foc_json_files:
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                try:
                    chunks = chunk_recipe_json(data, file_path)
                    all_chunks.extend(chunks)
                    recipe_count += 1
                except:
                    chunks = chunk_json_simple(file_path)
                    all_chunks.extend(chunks)

            except Exception as e:
                print(f"  ✗ Error loading {os.path.basename(file_path)}: {e}")

        print(f"✓ Loaded {recipe_count} recipes")
    else:
        print(f"✗ foc folder not found at {foc_folder}")

    # Load all CSV files (one chunk per file)
    print("\nLoading CSV files...")
    csv_files = glob.glob(os.path.join(processed_data_folder, '*.csv'))
    csv_count = 0

    for csv_file in csv_files:
        csv_chunks = chunk_csv_simple(csv_file)
        if csv_chunks:
            all_chunks.extend(csv_chunks)
            csv_count += 1
            print(f"✓ Loaded {os.path.basename(csv_file)}")

    # Load ingredient info from flavor network
    print("\nLoading ingredient pairing data...")
    pairing_file = os.path.join(processed_data_folder, 'ingredient_data.json')

    try:
        with open(pairing_file, 'r') as f:
            data = json.load(f)
        pairing_chunks = chunk_ingredient_data_json(data, pairing_file)
        all_chunks.extend(pairing_chunks)
        print(f"✓ Loaded {len(pairing_chunks)} ingredient pairing chunks")
    except Exception as e:
        print(f"✗ Error loading ingredient data: {e}")

    # too much memory -- maybe i can try to get it to be shorter
    # # Load cuisine-ingredient data
    # print("\nLoading cuisine-ingredient data...")
    # cuisine_json_path = os.path.join(processed_data_folder, "cuisine_ingredient_data.json")

    # with open(cuisine_json_path, 'r', encoding='utf-8') as f:
    #     cuisine_json = json.load(f)

    #     # Chunk the parsed data
    #     cuisine_chunks = chunk_cuisine_ingredients_dict(cuisine_json, cuisine_json_path)
    #     all_chunks.extend(cuisine_chunks)
    #     print(f"✓ Loaded {len(cuisine_chunks)} cuisine data chunks")


    # Load Reddit JSON files
    print("\nLoading Reddit JSON files...")
    reddit_folder = os.path.join(processed_data_folder, 'reddit')
    reddit_files = glob.glob(os.path.join(reddit_folder, '*.json'))
    reddit_count = 0

    for reddit_file in reddit_files:
        reddit_chunks = chunk_reddit_json(reddit_file)
        if reddit_chunks:
            all_chunks.extend(reddit_chunks)
            reddit_count += len(reddit_chunks)

    # Apply recursive chunking if requested
    if apply_chunking:
        print(f"\nApplying recursive text splitting...")
        original_count = len(all_chunks)
        all_chunks = apply_recursive_chunking(all_chunks)
        print(f"✓ Chunks after splitting: {len(all_chunks)} (was {original_count})")


    # Add all chunks to ChromaDB
    if all_chunks:
        print(f"\nAdding {len(all_chunks)} chunks to ChromaDB...")

        documents = [chunk['text'] for chunk in all_chunks]
        metadatas = [chunk['metadata'] for chunk in all_chunks]
        ids = [f"doc_{i}" for i in range(len(all_chunks))]

        # Single call - much simpler
        collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )

        print("✓ All documents added successfully!")

    return len(all_chunks)

In [41]:
# ===== QUERY & GENERATE =====

def query_rag(question, n_results=3):
    """Retrieve relevant chunks"""
    results = collection.query(
        query_texts=[question],
        n_results=n_results
    )
    return results['documents'][0], results['metadatas'][0]


def ask_cooking_assistant(question):
    """Main RAG function with Hugging Face"""
    # Retrieve relevant docs
    contexts, metadatas = query_rag(question, n_results=3)
    context_text = "\n\n---\n\n".join(contexts)

    # Build prompt
    prompt = f"""<|system|>
You are a helpful cooking assistant. Answer questions based on the provided context. Be concise and accurate.<|end|>
<|user|>
Context:
{context_text}

Question: {question}<|end|>
<|assistant|>"""

    # Generate response
    result = generator(
        prompt,
        max_new_tokens=300,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        return_full_text=False
    )

    answer = result[0]['generated_text'].strip()
    sources = list(set([m['source'].split('/')[-1] for m in metadatas]))

    return answer, sources

In [ ]:
if __name__ == "__main__":
    # Ingest all documents
    print("\n" + "="*60)
    print("COOKING ASSISTANT RAG SYSTEM - HUGGING FACE")
    print("="*60 + "\n")

    print("Step 1: Ingesting documents...")
    total_chunks = ingest_all_documents(collection)

    print("\n" + "="*60)
    print("System Ready! Ask cooking questions.")
    print("="*60 + "\n")


COOKING ASSISTANT RAG SYSTEM - HUGGING FACE

Step 1: Ingesting documents...
Loading core cooking technique files...
✓ Loaded 32 chunks from FCS.json
✓ Loaded 15 chunks from healthy_cooking_method.json

Loading recipes from foc folder...
✓ Loaded 33 recipes

Loading CSV files...
✓ Loaded cooking_methods.csv

Loading ingredient pairing data...
✓ Loaded 1171 ingredient pairing chunks

Loading cuisine-ingredient data...


In [34]:
    # Example queries
    test_questions = [
        "What is truffle? what tools can I use with the ingredient?"
    ]

    for question in test_questions:
        print(f"\n{'='*60}")
        print(f"Q: {question}")
        print(f"{'='*60}")

        answer, sources = ask_cooking_assistant(question)

        print(f"\nA: {answer}")
        print(f"\n📚 Sources: {', '.join(sources)}")


Q: What is truffle? what tools can I use with the ingredient?

A: The given text mentions a hand-held device called an immersion blender that can quickly blend a smoothie, a soup, a sauce, or a salad dressing. It is not mentioned whether the given text also includes a single-serve blender, but it's implied by the context that such a tool would be used for preparing one serving of a smoothie or shake.

📚 Sources: FCS.json, AskBaking_data.json


In [ ]:
question = "what kind of sauce is make with emulsion?"

answer, sources = ask_cooking_assistant(question)

print(f"\nA: {answer}")
print(f"\n📚 Sources: {', '.join(sources)}")


A: Answer: Espagnole sauce, which is a cold emulsion sauce made using carrot, beef stock, chopped onion, tomato puree, garlic, celery, butter, black pepper, flour, bay leaves, and prepared in a pan over low heat for 15 minutes until thickened.

📚 Sources: foc_ingredients_Espagnole_sauce.json, foc_ingredients_Oriental_sauce.json, foc_sections.json
